## Hyper Parameter Tuning

- In contrast to __model parameters__ which are learned during training, __model hyperparameters__ are set by the data scientist ahead of training and control implementation aspects of the model. 
- The __weights learned during training__ of a linear regression model are __parameters__ while the __number of trees in a random forest is a model hyperparameter__ because this is set by the data scientist. 
- __Hyperparameters__ can be thought of as __model settings__. These settings need to be tuned for each problem because the best model hyperparameters for one particular dataset will not be the best across all datasets. 
- The process of hyperparameter tuning (also called __hyperparameter optimization)__ means finding the combination of hyperparameter values for a machine learning model that performs the best - as measured on a validation dataset - for a problem.

In [ ]:
##! pip freeze

In [ ]:
##! pip install -U scikit-learn

### Hyper Parameter Tuning using RandomForest Classifier

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np

##### Scikit Learn modules needed for Logistic Regression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder,MinMaxScaler , StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Plotting libraries
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline

### Load training set for building the model

In [2]:
df = pd.read_csv("titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
print(df.describe())
df.isna().sum()

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### There are several blank values in few columns as seen above

### Manage blank values with imputation and build ML pipeline

I have createda sample Pipeline with few variables however I will suggest you to explore more and build your own pipelines

In [38]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['Age', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_features = ['Embarked', 'Sex', 'Pclass','SibSp']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state= 42))])

In [6]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df[['Fare','Pclass', 'Name', 'Sex', 'Age','Embarked','SibSp']],
                                                    df["Survived"], test_size=0.2,random_state = 42)

In [43]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.793


Reference on Grid Search
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

In [9]:
RandomForestClassifier()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [49]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__criterion': ["gini","entropy"],
    #'classifier__max_features': ["auto","sqrt","log2"],
    'classifier__max_depth':[10,50,100],
    'classifier__n_estimators':[10,50,150,200]
}

grid_search = GridSearchCV(clf, param_grid, cv=10, iid=False,n_jobs= -1, verbose= 2)
grid_search.fit(X_train, y_train)

print(("best Random Forest from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 193 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   43.4s finished


best Random Forest from grid search: 0.827


In [50]:
grid_search.best_params_

{'classifier__criterion': 'entropy',
 'classifier__max_depth': 10,
 'classifier__n_estimators': 200,
 'preprocessor__num__imputer__strategy': 'median'}

In [26]:
grid_search.best_params_

{'classifier__criterion': 'gini',
 'classifier__max_depth': 10,
 'classifier__n_estimators': 50,
 'preprocessor__num__imputer__strategy': 'mean'}

### Loading Test data
- Predicting on test set
- Please note that columns selected for training data must also be included in Testing data for prediction, else you will get error
- creating a submission file and extracting on disk
- submission file will be used for kaggle competition

In [35]:
test_data = pd.read_csv("titanic/test.csv")
X_test = test_data[["Pclass","Sex","Age","Fare","Embarked"]]
y_predicted = grid_search.predict(X_test)
y_predicted = pd.DataFrame({"Survived":y_predicted})

In [37]:
y_predicted.to_csv("My_submission_rf.csv",index=False)